# Problem 1: Data from yfinance

Importing the [yfinance](https://github.com/ranaroussi/yfinance) package.

In [2]:
# Import yfinance library
import yfinance as yf

# https://docs.python.org/3/library/datetime.html#examples-of-usage-date
import datetime as dt

Getting each data in variables.

In [3]:
# yf.download('META', 'AAPL', 'AMZN', 'NFLX', 'GOOG', period='5d')
#meta_data = yf.download('META', period='5d')
# apple_data = yf.download('AAPL', period='5d')
# amazon_data = yf.download('AMZN', period='5d')
# netflix = yf.download('NFLX', period='5d')
# google_data = yf.download('GOOG', period='5d')

# Using `download` method to get data for 5 days period
# https://ranaroussi.github.io/yfinance/reference/api/yfinance.download.html#yfinance-download
df = yf.download(['META', 'AAPL', 'AMZN', 'NFLX', 'GOOG'], period='5d')

/tmp/ipykernel_2808/2334644856.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(['META', 'AAPL', 'AMZN', 'NFLX', 'GOOG'], period='5d')
[*********************100%***********************]  5 of 5 completed
[*********************100%***********************]  5 of 5 completed


In [4]:
print(df)

Price            Close                                                  \
Ticker            AAPL        AMZN        GOOG        META        NFLX   
Date                                                                     
2025-11-14  272.410004  234.690002  276.980011  609.460022  111.217003   
2025-11-17  267.459991  232.869995  285.600006  602.010010  110.290001   
2025-11-18  267.440002  222.550003  284.959991  597.690002  114.089996   
2025-11-19  268.559998  222.690002  292.989990  590.320007  110.000000   
2025-11-20  268.559998  219.744995  295.500000  589.104980  107.029999   

Price             High                                                  ...  \
Ticker            AAPL        AMZN        GOOG        META        NFLX  ...   
Date                                                                    ...   
2025-11-14  275.959991  238.729996  279.149994  613.679993  114.273003  ...   
2025-11-17  270.489990  234.600006  294.519989  611.690002  111.849998  ...   
2025-11-18  

In [5]:
df.to_csv('data/data.csv')

In [6]:
# Current date and time
now = dt.datetime.now()

# Show.
now

datetime.datetime(2025, 11, 20, 18, 26, 15, 269605)

In [7]:
# Format: YYYYMMDD-HHMMSS
now.strftime("%Y%m%d-%H%M%S")

'20251120-182615'

In [8]:
# Create file name with timestamp
file_name = now.strftime("%Y%m%d-%H%M%S") + ".csv"

print(file_name)

20251120-182615.csv


In [9]:
# Save with timestamped filename
df.to_csv("data/" + now.strftime("%Y%m%d-%H%M%S") + ".csv")

# Problem 2: Plotting Data

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd

In [10]:
def plot_data():
    pass